<h1>Introduction</h1>
This analysis looks at different angles of the fuel data for the vehicle spreadsheet provided for the project. It looks at fuel in the scope of transmission and vehicle types and looks at the average price for different products.

These are the different libraries that are necessary to complete the analysis.

In [108]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
from math import factorial as ft 
import plotly.express as px

This reads the data into a DataFrame and displays the information and the first five rows of the data.

In [109]:
car_data = pd.read_csv('/Users/leahdeyoung/Desktop/GitHub/car-data-project-practicum/vehicles_us.csv', encoding = "utf-8")

display(car_data.head())
car_data.info()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


This is the beginning of cleaning the data. Model year is missing data, so I filled the missing data with the median year and converted datatype to datetime

In [110]:
print(car_data['model_year'].isna().sum())
year_median = car_data['model_year'].median()
year_median = round(year_median, 0)
car_data['model_year'] = car_data['model_year'].where((car_data['model_year'].notna()) , year_median) 
car_data['model_year'] = pd.to_datetime(car_data['model_year'], format='%Y')
car_data['model_year'] = car_data['model_year'].dt.year
print(car_data['model_year'].isna().sum())

3619
0


If paint color was not listed, I added the string "Unknown".

In [111]:
print(car_data['paint_color'].isna().sum())
car_data['paint_color'] = car_data['paint_color'].fillna('Unknown')
print(car_data['paint_color'].isna().sum())

9267
0


If the number of cylinders was blank, I filled it in with the median. I also converted the datatype to int.

In [112]:
print(car_data['cylinders'].isna().sum())
cylinder_median = car_data['cylinders'].median()
car_data['cylinders'] = car_data['cylinders'].where((car_data['cylinders'].notna()) , cylinder_median) 
car_data['cylinders'] = car_data['cylinders'].astype('int')
print(car_data['cylinders'].isna().sum())

5260
0


Odometer was also missing data, so I filled the missing data with 0 if the condition was "new" and converted datatype to int

In [113]:
print(car_data['odometer'].isna().sum())
odometer_mean = car_data.groupby('model_year')['odometer'].mean()
odometer_mean = odometer_mean.fillna(0)
odometer_mean = odometer_mean.reset_index().rename(columns={0: 'year', 'odometer': 'mean_odometer'})
car_data = car_data.merge(odometer_mean, on='model_year', how='left')
car_data['odometer'] = car_data['odometer'].fillna(car_data['mean_odometer'])
car_data = car_data.drop(columns='mean_odometer')
print(car_data['odometer'].isna().sum())

7892
0


This value is a 0 or 1 for true or false. I set the missing values to 0 to assume that if the value was missing, the car did not have four-wheel drive. Ultimately, I decided not to use this column in my analysis because of the uncertainty.

In [114]:
print(car_data['is_4wd'].isna().sum())
car_data['is_4wd'] = car_data['is_4wd'].fillna(0)
car_data['is_4wd'] = car_data['is_4wd'].astype('int')
print(car_data['is_4wd'].isna().sum())

25953
0


Checking to see that the DataFrame looks okay and that the information looks correct after filling in missing values and converting datatypes.

In [115]:
display(car_data.sample(10))
car_data.info()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
39900,23900,2014,ford f350,excellent,8,diesel,140000.000000,automatic,truck,white,1,2018-08-01,28
30835,23990,2014,ram 1500,good,8,gas,93699.084308,automatic,truck,silver,1,2019-03-15,29
50083,13900,2013,ford explorer,excellent,6,gas,103403.558132,automatic,SUV,Unknown,1,2018-07-25,51
21209,11905,2008,gmc sierra 1500,good,8,gas,146676.000000,automatic,truck,blue,0,2018-05-08,37
41270,14998,2014,chrysler 300,excellent,8,gas,89000.000000,automatic,sedan,grey,1,2018-07-18,3
41561,20500,2015,ford explorer,excellent,6,gas,76000.000000,automatic,SUV,grey,1,2018-11-30,9
28193,6700,2012,chevrolet equinox,good,6,gas,111000.000000,automatic,SUV,white,1,2019-01-15,5
14012,4000,2005,subaru forester,good,4,gas,151960.236054,automatic,SUV,red,1,2018-11-26,25
47009,2200,1994,ford f350,fair,8,diesel,250000.000000,automatic,truck,Unknown,1,2018-07-31,36
14896,2200,2003,chevrolet impala,good,6,gas,218000.000000,automatic,sedan,silver,0,2019-02-16,30


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  int64  
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  int64  
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  int64  
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 5.5+ MB


Checking different values for fuel, transmission, and vehicle type to determine if there are duplicates and if analysis on these columns is a viable option.

In [116]:
print(car_data['fuel'].unique())
print(car_data['transmission'].unique())
print(car_data['type'].unique())

['gas' 'diesel' 'other' 'hybrid' 'electric']
['automatic' 'manual' 'other']
['SUV' 'pickup' 'sedan' 'truck' 'coupe' 'van' 'convertible' 'hatchback'
 'wagon' 'mini-van' 'other' 'offroad' 'bus']


Grouped data by transmission and fuel and then counted it. Created histograms for both. This sets up a scatter plot later.

In [117]:

car_fuel = car_data.groupby('fuel')['fuel'].count()
car_transmission = car_data.groupby('transmission')['transmission'].count()
car_fuel_hist = px.histogram(car_fuel, 
                             x='fuel', 
                             nbins=5, 
                             title='Fuel and Transmission Type Frequency')
car_trans_hist = px.histogram(car_transmission, 
                             x='transmission', 
                             nbins=10,
                             title='Transmission Type Frequency' 
                             )
car_fuel_hist.show()
car_trans_hist.show()

Grouped vehicle data by vehicle type and then counted it. This sets up the second scatter plot later.

In [118]:
car_type_frequency = car_data.groupby('type')['type'].count()
car_type_hist = px.histogram(car_type_frequency, 
                             y='type', 
                             nbins=8, 
                             title='Car Type Frequency')
car_type_hist.show()

This groups data by fuel and transmission, then calculates the average price. Then I created a new dataframe out of the series, and created a scatter plot.

In [119]:
grp = car_data.groupby(['fuel', 'transmission'])
car_fuel_transmission = grp['price'].mean()
car_fuel_transmission = car_fuel_transmission.reset_index().rename(columns={0: 'price'})
#price_fuel = car_data.groupby('fuel')['price'].mean()
car_fuel_scatter = px.scatter(car_fuel_transmission,
                              x='fuel',
                              y='price',
                              color='transmission',
                              labels={
                                'price': 'Average Price',
                                'fuel': 'Fuel Type',
                                'transmission': 'Transmission Type'},                              
                              title='Average Price per Type of Transmission and Fuel')
car_fuel_scatter.show()

This groups the data by fuel and vehicle type, then calculates average price. Then the series is put into a dataframe, and a scatter plot is created. 

In [120]:
grp2 = car_data.groupby(['fuel', 'type'])
car_type = grp2['price'].mean()
car_type = car_type.reset_index().rename(columns={0: 'price'})
car_type_scatter = px.scatter(car_type,
                              x='type',
                              y='price',
                              color='fuel',
                              labels={
                                  'price': 'Average Price',
                                  'type': 'Vehicle Type',
                                  'fuel': 'Fuel Type'},
                              title='Average Price for Vehicle and Fuel Type')
car_type_scatter.show()

<h1>Conclusion</h1>
Disregarding the "other" data, diesel seems to be the more expensive options when it comes to fuel, and larger vehicles seem to be more expensive than smaller vehicles. Fuel type seems to have more impact on price than transmission type. Overall, I would not recommend basing a purchase on this data alone, but I think it is good to take it into account.